# Who is heading for Diabetes?

diabetesahead

This is the predictive part of the 2017 Melbourne Datathon.

The task is to predict the probability that a patient will be dispensed a drug related to Diabetes post 2015. This is quite important research as it will be an early warning system for doctors so intervention can potentially be made before it is too late.

Use the patients that we have provided all the records for to build your model, then see how it performs on these unseen people.

For patient ID'S 279,201 to 558,352 you need to submit a file with 2 columns, the Patient_ID and the probability in the range [0-1]. The file will have 279,153 rows including the header row. An example submission file is provided for download.

In [178]:
import pandas as pd
import numpy as np
import sqlite3

from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn import svm

In [138]:
submission = pd.read_csv('../../submissions/diabetes_submission_example.csv')  

## Load the lookup data

In [139]:
conn = sqlite3.connect("../../sql/datasci.db")

In [152]:
def patient_data(conn, patient_id):
    """
    Return the patient data.
    """
    SQL = """
SELECT *
FROM transactions a
LEFT OUTER JOIN ChronicIllness_LookUp b 
    ON a.Drug_ID = b.MasterProductID 
LEFT OUTER JOIN patients c
    ON a.Patient_ID = c.Patient_ID
LEFT OUTER JOIN classification d
    ON a.Patient_ID = d.Patient_ID
WHERE a.Patient_ID = {}
    """.format(patient_id)

    return pd.read_sql_query(SQL, conn)

## Compute some basic features of the data 

In [176]:
gender_map = {'F': 1, 'M': 0, 'U': 0.5}

def feature_extract(patient_frame):
    """
    The thing that forms a feature vector.
    
    ** Make sure to partition out data from the 2016 period.
    
    """
    
    x = [gender_map[patient_frame.gender[0]], 2016 - patient_frame.year_of_birth[0]]
    y = patient_frame.Target[0]
    
    return x, y


## Look at the descriptive power of the features.


Look at the relationship of variables and plot them.

## Create the matrix of features and vector of targets

In [ ]:
X, Y = [], []
for i in range(1, 10000): 
    x, y = feature_extract(patient_data(conn, i))
    X.append(x)
    Y.append(y)

In [ ]:
X = np.vstack(X)
y = np.array(Y)

## Fit a support vector machine to the data, using 5 fold cross validation.

In [ ]:
clf = svm.SVC(kernel='rbf', C=1)

scores = cross_val_score(clf, X, y, cv=5)
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Classify the remaining dataset anf form a submission